In [1]:
!pip install pytorch-lightning -qqq

In [2]:
import numpy as np
import pandas as pd
import torch # торч
from torch import nn # модуль с базовыми слоями
import torch.nn.functional as F # модуль с базовыми функциями

In [3]:
import random
from numpy.random import seed
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

random.seed(69)
seed(69)
torch.manual_seed(69)
seed_everything(69, workers=True)

INFO:lightning_lite.utilities.seed:Global seed set to 69


69

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

### **Задача 1**. Посчитайте количество обучаемых параметров в сети net_seq и net_model.

In [5]:
net_seq = nn.Sequential(
    nn.Linear(in_features=3, out_features=5),
    nn.Sigmoid(),
    nn.Linear(in_features=5, out_features=2),                                                            
)
net_seq = net_seq.to(device)
net_seq

Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=5, out_features=2, bias=True)
)

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=3, out_features=5)
        self.fc2 = nn.Linear(in_features=5, out_features=2)

    def forward(self, x):
        x = self.fc1(x)
        return F.sigmoid(self.fc2(x))

net_model = Model()
net_model.to(device)
net_model

Model(
  (fc1): Linear(in_features=3, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=2, bias=True)
)

Кол-во обучаемых параметров = кол-во весов + кол-во смещений

$K = (in * mid + mid * out) + (mid + out)$

где K - кол-во обучаемых параметров,
in - кол-во входных признаков,
mid - кол-во нейронов в скрытом слое,
out - кол-во нейронов в выходном слое

In [7]:
def calc_neurons(model):
  layers = list(model.parameters())
  in_ = layers[0].shape[1]
  mid = layers[1].shape[0]
  out = layers[2].shape[0]
  K = in_*mid+mid*out+mid+out
  return K

print('Кол-во обучаемых параметров в net_model', calc_neurons(net_model))
print('Кол-во обучаемых параметров в net_seq', calc_neurons(net_seq))

Кол-во обучаемых параметров в net_model 32
Кол-во обучаемых параметров в net_seq 32


## **Задача 2**. Cделать нейрон, соответствующий оператору НЕ.

In [8]:
class NotNeuron(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = torch.nn.Linear(1, 1)
    self.fc.weight.data = torch.tensor([[-228.]])
    self.fc.bias.data = torch.tensor([69.0])

  def forward(self, x):
    return torch.heaviside(self.fc(x), torch.tensor([0.0]))

In [9]:
neuron = NotNeuron()
neuron.fc.weight, neuron.fc.bias

(Parameter containing:
 tensor([[-228.]], requires_grad=True), Parameter containing:
 tensor([69.], requires_grad=True))

In [10]:
def test_one_var_neuron(neuron):
  tests = [[0.0], [1.0]]
  for test in tests:
    x = torch.tensor(test)
    res = neuron(x).detach().numpy()[0]
    print('input:', test, 'output:', res)

In [11]:
test_one_var_neuron(neuron)

input: [0.0] output: 1.0
input: [1.0] output: 0.0


## **Задача 3**. Cделать нейрон, соответствующий оператору И.

In [12]:
class TruthTable:
  """
  Class to build beautiful truth table
  """
  def __init__(self, vars):
    self.vars = vars
  
  def __beautiful_binary(self, i):
    i_bin = bin(i)[2:] # Convert index to bin
    i_bin_formatted = ("{:0"+str(self.vars)+"d}").format(int(i_bin))  # Format to 000-001-010-011 etc
    return i_bin_formatted

  def __iterate_neuron(self, neuron):
    total_inputs = []
    total_outputs = []
    for i in range(2**self.vars):
      # string of inputs
      i_bin_formatted = self.__beautiful_binary(i)
      # convert input to tensor
      raw_inputs = []
      for c in i_bin_formatted:
        raw_inputs.append(float(c))
      inp = torch.tensor(raw_inputs)

      # calc results
      out = neuron(inp).detach().numpy()

      # save results
      total_inputs.append(raw_inputs)
      total_outputs.append(out)
    
    return total_inputs, total_outputs
  
  def __fill_dict(self, d, key_str, array):
    """
    Create indexed keys in dict to make columns in df
    """
    for inp in array:
      for i in range(len(inp)):
        t_key = key_str+str(i)
        t_value = inp[i]

        if t_key in d:
          d[t_key].append(t_value)
        else:
          d[t_key] = [t_value]
  
  def build_table(self, neuron) -> pd.DataFrame:
    inputs, outputs = self.__iterate_neuron(neuron)

    data = dict()
    self.__fill_dict(data, 'inp', inputs)
    self.__fill_dict(data, 'out', outputs)

    df = pd.DataFrame(data)
    return df

In [13]:
class NeuronAND(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = torch.nn.Linear(2, 1)
    self.fc.weight.data = torch.tensor([[69., 228.0]])
    self.fc.bias.data = torch.tensor([-228.0])

  def forward(self, x):
    return torch.heaviside(self.fc(x), torch.tensor([0.0]))

In [14]:
neuron = NeuronAND()
neuron.fc.weight, neuron.fc.bias

(Parameter containing:
 tensor([[ 69., 228.]], requires_grad=True), Parameter containing:
 tensor([-228.], requires_grad=True))

In [15]:
tt = TruthTable(2)
table = tt.build_table(neuron)
table

,inp0,inp1,out0
0,0.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,1.0,1.0


## **Задача 4**. Cделать нейрон, соответствующий оператору ИЛИ.

In [16]:
class NeuronOR(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = torch.nn.Linear(2, 1)
    self.fc.weight.data = torch.tensor([[228., 228.0]])
    self.fc.bias.data = torch.tensor([-69.0])

  def forward(self, x):
    return torch.heaviside(self.fc(x), torch.tensor([0.0]))

In [17]:
neuron = NeuronOR()
neuron.fc.weight, neuron.fc.bias

(Parameter containing:
 tensor([[228., 228.]], requires_grad=True), Parameter containing:
 tensor([-69.], requires_grad=True))

In [18]:
table = tt.build_table(neuron)
table

,inp0,inp1,out0
0,0.0,0.0,0.0
1,0.0,1.0,1.0
2,1.0,0.0,1.0
3,1.0,1.0,1.0


## **Задача 5**. Cделать нейрон, соответствующий оператору XOR.

In [19]:
class NeuronXOR(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # use OR neuron
    self.nOR = NeuronOR()
    # use NOT neuron
    self.nNOT = NotNeuron()
    # use and neuron
    self.nAND = NeuronAND()


  def forward(self, x):
    OR_res = self.nOR(x)
    AND_res = self.nAND(x)
    NOT_AND_res = self.nNOT(AND_res)
    
    _temp = torch.cat((OR_res, NOT_AND_res), 0) # Concat (OR, NOT_AND)
    FINAL_res = self.nAND(_temp) #  AND(OR, NOT_AND) = XOR
    return torch.heaviside(FINAL_res, torch.tensor([0.0]))

In [20]:
table = tt.build_table(NeuronXOR())
table

,inp0,inp1,out0
0,0.0,0.0,0.0
1,0.0,1.0,1.0
2,1.0,0.0,1.0
3,1.0,1.0,0.0


## **Вопрос 1**. Какие нейронные сети могут иметь только линейную разделяющую поверхность?

1. Нейросети без функции активации
2. Один нейрон с пороговой функцией активации

## **Вопрос 2**. Имеет ли смысл соединять полносвязанные нейроны (нейроны, которые принимают на вход все выходы предыдущего слоя) с линейной функцией активации в многослойную нейронную сеть?

Не имеет.

Подробный ответ дан в [закреплённом комментарии](https://stepik.org/lesson/210102/step/3?discussion=1146539&unit=183579) 😄

**Домашнее задание 1:** реализуйте XOR с помощью 3 нейронов. Запишите ответ в виде выражения, состоящего из объектов neuron() – моделей нейрона с пороговой функцией активации, внутри скобок может быть что угодно. Входы верхнего уровня называются x1 и x2. Пример фрагмента записи: neuron(1*x1 + 5*x2 - 0.1) + neuron(x1) (ответ будет выглядеть чуть сложнее, но других символов вроде && не потребуется).

neuron(neuron(1*x1+1*x2-0.5)-neuron(0.1*x1+1*x2-1)+0)

[Вот подсказка](https://ucarecdn.com/271cb97b-58a9-414f-adb9-745e397d87fc/)

**Домашнее задание 2:** нарисуйте backward граф для выражения `a*b+c*d`. [Теория и пример оформления](https://www.youtube.com/watch?v=MswxJw-8PvE). Сравните полученные теоретические значения с аттрибутами grad у исходных тензоров.

In [21]:
a = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([4.0], requires_grad=True)
c = torch.tensor([1.0], requires_grad=True)
d = torch.tensor([5.0], requires_grad=False)

In [22]:
print('a.grad', a.grad)
print('b.grad', a.grad)
print('c.grad', a.grad)
print('d.grad', a.grad)

a.grad None
b.grad None
c.grad None
d.grad None


In [23]:
d1 = a*b
d2 = c*d
loss = d1 + d2
loss.backward()

![](https://lh5.googleusercontent.com/1B87ZJo0WWgzYu7EDMI6cAae4ZeZn7sujtXLTr--wWG7xM8jOQSjKHWQWkkRfJuGJpg=w2400)


[Исходник схемы](https://drive.google.com/file/d/1V5AVg7A4L7y3pGJpqD3p35F7QJYtyb6H/view?usp=share_link)

In [24]:
print('a.grad', a.grad)
print('b.grad', b.grad)
print('c.grad', c.grad)
print('d.grad', d.grad)

a.grad tensor([4.])
b.grad tensor([2.])
c.grad tensor([5.])
d.grad None


**Домашнее задание 3:** Поэксперементируйте с размером тензоров, которые влезут на видеоркарту в Colab. Найдите максимальный размер тензора для типа данных float32, float64, float16, int32, int64. На сколько они отличаются.

In [25]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)

In [26]:
!nvidia-smi

Sun Dec  4 09:42:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |    612MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
def allocate_empty_tensor(a_size, dim_size, type_):
  a=torch.zeros(a_size,dim_size,dtype=type_,device='cuda')

In [28]:
def allocate_and_check(a_size, dim_size, type_):
  allocate_empty_tensor(a_size, dim_size, type_)
  print('allocated:',torch.cuda.memory_allocated())
  print('reserved:',torch.cuda.memory_reserved())

Были найдены следующие максимумы

In [29]:
 torch.cuda.empty_cache()

In [30]:
 allocate_and_check(4096, 927610, torch.float32)
 torch.cuda.empty_cache()

allocated: 4096
reserved: 15200157696


In [31]:
 allocate_and_check(4096, 463805, torch.float64)
 torch.cuda.empty_cache()

allocated: 4096
reserved: 15200157696


In [32]:
 allocate_and_check(4096, 927610, torch.int32)
 torch.cuda.empty_cache()

allocated: 4096
reserved: 15200157696


In [33]:
 allocate_and_check(4096, 463805, torch.int64)
 torch.cuda.empty_cache()

allocated: 4096
reserved: 15200157696


Размеры тензоров для 64-битных типов в 2 раза меньше, чем для 32-битных. Логично, ведь 64-битные должны занимать в 2 раза больше памяти.

**Домашнее задание 4:** Напишите хороший пример неэффективного кода для занятия памяти видеокарты, который вызовет ошибку out of memory

In [34]:
try:
  torch.zeros(4096, 927605, dtype=torch.int64,device='cuda') + torch.ones(1, 1, dtype=torch.int64,device='cuda')
except Exception as e:
  print(e)

CUDA out of memory. Tried to allocate 28.31 GiB (GPU 0; 14.76 GiB total capacity; 4.00 KiB already allocated; 14.16 GiB free; 2.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


In [35]:
 torch.cuda.empty_cache()

**Домашнее задание 5:** Используя один линейный слой `nn.Linear` и один входной тензор `x` подберите подберите размерности так, чтобы занимать всю видеопамять.
Попробуйте применить линейный слой к тензору `x`. Что произойдет? Кратко опишите ваши эксперименты. Что вы поняли?

In [36]:
class NotNeuronGIGACHAD(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = torch.nn.Linear(1, 200800).to(device)
    self.fc.weight.data = torch.tensor([[-228.]]).to(device)
    self.fc.bias.data = torch.tensor([69.0]).to(device)

  def forward(self, x):
    return torch.heaviside(self.fc(x), torch.tensor([0.0]).to(device))

In [37]:
torch.cuda.empty_cache()
print('allocated:',torch.cuda.memory_allocated())
print('reserved:',torch.cuda.memory_reserved())

allocated: 4096
reserved: 2097152


In [38]:
torch.cuda.empty_cache()
print('allocated:',torch.cuda.memory_allocated())
print('reserved:',torch.cuda.memory_reserved())

lmao = torch.zeros(4096*200800,1,dtype=torch.float32,device='cuda')
neuron = NotNeuronGIGACHAD()
neuron(lmao)

allocated: 4096
reserved: 2097152


tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<NotImplemented>)

В результате экспериментов стало понятно, что если ячейка завершается с ошибкой, то выполнение torch.cuda.empty_cache() не очищает память и приходится перезапускать среду =(

### Основной вывод
Если создать нейрон и данные для него, которые займут всю память, то станет невозможно использовать этот нейрой, т.к. при вызове forward происходит ещё одно выделение памяти, что приводит к out-of-memory (OOM).

Успешное выполнение происходит, если занять < 50% памяти, что сделано в ячейке выше. 

В ячейке ниже занимается примерно 50(в данных) + 25(в нейроне) = 75% общей памяти GPU, в этом случае происходит OOM. Видимо, в функции forward происходит копирование данных, поэтому необходимо иметь 50% свободной памяти для гарантии выполнения функции.

In [39]:
torch.cuda.empty_cache()
print('allocated:',torch.cuda.memory_allocated())
print('reserved:',torch.cuda.memory_reserved())

lmao = torch.zeros(4096*463805,1,dtype=torch.float32,device='cuda')
neuron = NotNeuronGIGACHAD()
neuron(lmao)

allocated: 6580868096
reserved: 6582960128


RuntimeError: ignored